In [ ]:
%cd '/content/drive/MyDrive/8th-semester/IR/IR_Project_W23/src'

/content/drive/MyDrive/8th-semester/IR/IR_Project_W23/src


In [ ]:
!pip install transformers
!pip3 install git+https://github.com/ernie-mlg/rpunct.git
!pip install git+https://github.com/tagucci/pythonrouge.git
# from tqdm.auto import tqdm
from tqdm import trange, tqdm
import requests
import json
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 83.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 98.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 27.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/ernie-mlg/rpunct.git to /tmp/pip-req-build-vfkmj3c8
  Running command git clone --filter=blob:none --quiet https://github.com/ernie-mlg/rpunct.git /tmp/pip-req-build-vfkmj3c8
  Resolved https://github.com/ernie-mlg/rpunct.git to commit 4c378cf570f05aa58c7d3dbb896b51ca2d95fde6
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 KB 37.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 96.7 MB/s eta 0:00:00
     ━━━━━━━━━━

In [ ]:
data_file = '/content/drive/MyDrive/8th-semester/IR/IR_Project_W23/data/cnn10.json'
with open(data_file) as f:
    video_data = json.load(f)
f.close()

df = pd.read_json(data_file)
df.head()

,id,text_captions,fused_captions,summary
0,8GgaCgjAFLM,[music] Thank you Hello lovely people Happy! F...,"[{'text': '[Music]', 'start': 0.0, 'duration':...",It's been nearly three years since the start ...
1,Kc_GMYHJNFQ,[music] Thank you hello everyone Wonderful! We...,"[{'text': '[Music]', 'start': 0.0, 'duration':...","Some farmers are not harvesting any tomatoes,..."
2,wPdiuj2JwEU,[music] Thank you Hey there everyone Terrific!...,"[{'text': '[Music]', 'start': 0.0, 'duration':...",CNN Business Correspondent Rahel Solomon has ...
3,H07TJT3VpSY,"[music] Thank you! What's up Sunshine, Welcome...","[{'text': '[Music]', 'start': 0.0, 'duration':...",Friday marked one year since Russian Presiden...
4,U4WxR7hXfCQ,[music] Thank you! What's up lovely people? Th...,"[{'text': '[Music]', 'start': 0.0, 'duration':...",Massive storm system pummeling much of the co...


In [ ]:
def summarize(tokenizer, model, text, device='cuda:0'):

    inputs = tokenizer(text, 
                    max_length=1024, 
                    truncation=True,
                    return_tensors="pt").to(device)
    model.to(device)
    summary_ids = model.generate(inputs["input_ids"], max_new_tokens=120, min_length=90)
    summary = tokenizer.batch_decode(summary_ids, 
                                  skip_special_tokens=True, 
                                  clean_up_tokenization_spaces=False)
    return summary

In [ ]:
checkpoint = "google/pegasus-large"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [ ]:
df['pegasus_summary'] = ""
for i in trange(len(df)):
    df['pegasus_summary'].iloc[i] = summarize(tokenizer, model, df['text_captions'].iloc[i])[0]

df.head()

100%|██████████| 10/10 [00:36<00:00,  3.61s/it]


,id,text_captions,fused_captions,summary,pegasus_summary,mT5_summary,pegasus_rouge
0,8GgaCgjAFLM,[music] Thank you Hello lovely people Happy! F...,"[{'text': '[Music]', 'start': 0.0, 'duration':...",It's been nearly three years since the start ...,it's been nearly three years since the pandemi...,This is a full transcript of the latest report...,None
1,Kc_GMYHJNFQ,[music] Thank you hello everyone Wonderful! We...,"[{'text': '[Music]', 'start': 0.0, 'duration':...","Some farmers are not harvesting any tomatoes,...",We're going to start with news on a vegetable ...,This is a full transcript of the BBC's weekly ...,None
2,wPdiuj2JwEU,[music] Thank you Hey there everyone Terrific!...,"[{'text': '[Music]', 'start': 0.0, 'duration':...",CNN Business Correspondent Rahel Solomon has ...,"We'll end with our typical 10 out of 10 story,...",This is a full transcript of The Next Frontier...,None
3,H07TJT3VpSY,"[music] Thank you! What's up Sunshine, Welcome...","[{'text': '[Music]', 'start': 0.0, 'duration':...",Friday marked one year since Russian Presiden...,"Our lead story on CNN 10 Today, the ongoing in...",This is a full transcript of the BBC's 10 Seco...,None
4,U4WxR7hXfCQ,[music] Thank you! What's up lovely people? Th...,"[{'text': '[Music]', 'start': 0.0, 'duration':...",Massive storm system pummeling much of the co...,We start with continuing coverage on the inten...,This is a full transcript of Brian Todd's cove...,None


In [ ]:
checkpoint = "csebuetnlp/mT5_multilingual_XLSum"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

df['mT5_summary'] = ""
for i in trange(len(df)):
    df['mT5_summary'].iloc[i] = summarize(tokenizer, model, df['text_captions'].iloc[i])[0]

df.head()

/usr/local/lib/python3.9/dist-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
100%|██████████| 10/10 [00:39<00:00,  3.99s/it]


,id,text_captions,fused_captions,summary,pegasus_summary,mT5_summary,pegasus_rouge
0,8GgaCgjAFLM,[music] Thank you Hello lovely people Happy! F...,"[{'text': '[Music]', 'start': 0.0, 'duration':...",It's been nearly three years since the start ...,it's been nearly three years since the pandemi...,This is a full transcript of the latest report...,None
1,Kc_GMYHJNFQ,[music] Thank you hello everyone Wonderful! We...,"[{'text': '[Music]', 'start': 0.0, 'duration':...","Some farmers are not harvesting any tomatoes,...",We're going to start with news on a vegetable ...,This is a full transcript of the BBC's weekly ...,None
2,wPdiuj2JwEU,[music] Thank you Hey there everyone Terrific!...,"[{'text': '[Music]', 'start': 0.0, 'duration':...",CNN Business Correspondent Rahel Solomon has ...,"We'll end with our typical 10 out of 10 story,...",This is a full transcript of The Next Frontier...,None
3,H07TJT3VpSY,"[music] Thank you! What's up Sunshine, Welcome...","[{'text': '[Music]', 'start': 0.0, 'duration':...",Friday marked one year since Russian Presiden...,"Our lead story on CNN 10 Today, the ongoing in...",This is a full transcript of the BBC's 10 Seco...,None
4,U4WxR7hXfCQ,[music] Thank you! What's up lovely people? Th...,"[{'text': '[Music]', 'start': 0.0, 'duration':...",Massive storm system pummeling much of the co...,We start with continuing coverage on the inten...,This is a full transcript of Brian Todd's cove...,None


In [ ]:
df['pegasus_summary'].iloc[0], df['mT5_summary'].iloc[0]

("it's been nearly three years since the pandemic began, and we're still trying to figure out answers about how exactly this virus was able to infect so many people and take so many lives. Reporting this week states that with quote low confidence, the U.S Department of Energy and the Director of the FBI endorse a lab leak theory meaning the virus accidentally escaped from a lab in Wuhan China Low confidence according to the office of the Director of National Intelligence generally means that the information is scant, questionable, or very fragmented, or that the intelligence Community has significant concerns about the sources.",
 "This is a full transcript of the latest reporting on the origins of Covid-19. Here's what you need to know about the virus and how it started to spread to humans. Welcome to Coy Wire - CNN chief investigative correspondent Pamela Brown, who has been in the US since the pandemic began, and explains why the BBC . BBC News ")

In [ ]:
df.describe()

,id,text_captions,fused_captions,summary,pegasus_summary,mT5_summary,pegasus_rouge
count,10,10,10,10,10,10,0
unique,9,9,9,9,9,9,0
top,rQp4putiL_A,[music] Thank you Hello lovely people! Wonderf...,"[{'text': '[Music]', 'start': 0.0, 'duration':...",CNN's Brian Todd tells us about President Bid...,Now we start today with more on President Bide...,This is a full transcript of President Joe Bid...,NaN
freq,2,2,2,2,2,2,NaN


In [ ]:
df.to_excel('../data/video_summary.xlsx', index=False)

# Evaluation

In [ ]:
eval_df = pd.read_excel('../data/video_summary.xlsx')
eval_df.head()

,id,text_captions,fused_captions,summary,pegasus_summary,mT5_summary,pegasus_rouge
0,8GgaCgjAFLM,[music] Thank you Hello lovely people Happy! F...,"[{'text': '[Music]', 'start': 0.0, 'duration':...",It's been nearly three years since the start ...,it's been nearly three years since the pandemi...,This is a full transcript of the latest report...,NaN
1,Kc_GMYHJNFQ,[music] Thank you hello everyone Wonderful! We...,"[{'text': '[Music]', 'start': 0.0, 'duration':...","Some farmers are not harvesting any tomatoes,...",We're going to start with news on a vegetable ...,This is a full transcript of the BBC's weekly ...,NaN
2,wPdiuj2JwEU,[music] Thank you Hey there everyone Terrific!...,"[{'text': '[Music]', 'start': 0.0, 'duration':...",CNN Business Correspondent Rahel Solomon has ...,"We'll end with our typical 10 out of 10 story,...",This is a full transcript of The Next Frontier...,NaN
3,H07TJT3VpSY,"[music] Thank you! What's up Sunshine, Welcome...","[{'text': '[Music]', 'start': 0.0, 'duration':...",Friday marked one year since Russian Presiden...,"Our lead story on CNN 10 Today, the ongoing in...",This is a full transcript of the BBC's 10 Seco...,NaN
4,U4WxR7hXfCQ,[music] Thank you! What's up lovely people? Th...,"[{'text': '[Music]', 'start': 0.0, 'duration':...",Massive storm system pummeling much of the co...,We start with continuing coverage on the inten...,This is a full transcript of Brian Todd's cove...,NaN


In [ ]:
!pip install rouge-metric
from rouge_metric import PyRouge

# Load summary results
hypotheses = [
    'how are you\ni am fine',  # document 1: hypothesis
    'it is fine today\nwe won the football game',  # document 2: hypothesis
]
references = [[
    'how do you do\nfine thanks',  # document 1: reference 1
    'how old are you\ni am three',  # document 1: reference 2
], [
    'it is sunny today\nlet us go for a walk',  # document 2: reference 1
    'it is a terrible day\nwe lost the game',  # document 2: reference 2
]]

# Evaluate document-wise ROUGE scores
rouge = PyRouge(rouge_n=(1, 2, 4), rouge_l=True, rouge_w=True,
                rouge_w_weight=1.2, rouge_s=True, rouge_su=True, skip_gap=4)
scores = rouge.evaluate(hypotheses, references)
print(scores)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
{'rouge-1': {'r': 0.5182186234817814, 'p': 0.5555555555555556, 'f': 0.5362379555927943}, 'rouge-2': {'r': 0.19518716577540107, 'p': 0.2125, 'f': 0.20347597966879813}, 'rouge-4': {'r': 0.07142857142857142, 'p': 0.08333333333333333, 'f': 0.07692307692307691}, 'rouge-l': {'r': 0.5182186234817814, 'p': 0.5555555555555556, 'f': 0.5362379555927943}, 'rouge-w-1.2': {'r': 0.33608419409971513, 'p': 0.4734837712933738, 'f': 0.3931242798550236}, 'rouge-s4': {'r': 0.2549450549450549, 'p': 0.2916666666666667, 'f': 0.27207237393198186}, 'rouge-su4': {'r': 0.3149522799575822, 'p': 0.35526315789473684, 'f': 0.3338954468802698}}


In [ ]:
rouge = PyRouge(rouge_n=(1, 2, 4), rouge_l=True, rouge_w=True,
                rouge_w_weight=1.2, rouge_s=True, rouge_su=True, skip_gap=4)

def compute_summary_rouge(pred, gt):
    scores = rouge.evaluate([pred,], [[gt,],])
    return scores

In [ ]:
eval_df['pegasus_rouge']=None
eval_df['mT5_rouge'] = None
pegasus_rouge_dict = []
mT5_rouge_dict = []

for i in trange(len(df)):
    temp = compute_summary_rouge(eval_df['pegasus_summary'].iloc[i], eval_df['summary'].iloc[i])
    pegasus_rouge_dict.append(temp)
    eval_df['pegasus_rouge'].iloc[i] = str(temp)

    temp = compute_summary_rouge(eval_df['mT5_summary'].iloc[i], eval_df['summary'].iloc[i])
    mT5_rouge_dict.append(temp)
    eval_df['mT5_rouge'].iloc[i] = str(temp)

eval_df.head()

100%|██████████| 10/10 [00:00<00:00, 11.82it/s]


,id,text_captions,fused_captions,summary,pegasus_summary,mT5_summary,pegasus_rouge,mT5_rouge
0,8GgaCgjAFLM,[music] Thank you Hello lovely people Happy! F...,"[{'text': '[Music]', 'start': 0.0, 'duration':...",It's been nearly three years since the start ...,it's been nearly three years since the pandemi...,This is a full transcript of the latest report...,"{'rouge-1': {'r': 0.38144329896907214, 'p': 0....","{'rouge-1': {'r': 0.24742268041237114, 'p': 0...."
1,Kc_GMYHJNFQ,[music] Thank you hello everyone Wonderful! We...,"[{'text': '[Music]', 'start': 0.0, 'duration':...","Some farmers are not harvesting any tomatoes,...",We're going to start with news on a vegetable ...,This is a full transcript of the BBC's weekly ...,"{'rouge-1': {'r': 0.226890756302521, 'p': 0.32...","{'rouge-1': {'r': 0.15126050420168066, 'p': 0...."
2,wPdiuj2JwEU,[music] Thank you Hey there everyone Terrific!...,"[{'text': '[Music]', 'start': 0.0, 'duration':...",CNN Business Correspondent Rahel Solomon has ...,"We'll end with our typical 10 out of 10 story,...",This is a full transcript of The Next Frontier...,"{'rouge-1': {'r': 0.208, 'p': 0.25242718446601...","{'rouge-1': {'r': 0.144, 'p': 0.32142857142857..."
3,H07TJT3VpSY,"[music] Thank you! What's up Sunshine, Welcome...","[{'text': '[Music]', 'start': 0.0, 'duration':...",Friday marked one year since Russian Presiden...,"Our lead story on CNN 10 Today, the ongoing in...",This is a full transcript of the BBC's 10 Seco...,"{'rouge-1': {'r': 0.38181818181818183, 'p': 0....","{'rouge-1': {'r': 0.10909090909090909, 'p': 0...."
4,U4WxR7hXfCQ,[music] Thank you! What's up lovely people? Th...,"[{'text': '[Music]', 'start': 0.0, 'duration':...",Massive storm system pummeling much of the co...,We start with continuing coverage on the inten...,This is a full transcript of Brian Todd's cove...,"{'rouge-1': {'r': 0.2698412698412698, 'p': 0.3...","{'rouge-1': {'r': 0.14285714285714285, 'p': 0...."


In [ ]:
eval_df.to_excel('../data/video_summary.xlsx', index=False)

In [ ]:
pegasus_rouge_df = pd.DataFrame.from_dict(pegasus_rouge_dict)

for column in pegasus_rouge_df:
    pegasus_rouge_df[column] = pegasus_rouge_df[column].apply(lambda x: x['f'])

pegasus_rouge_df.head()
pegasus_rouge_df.describe()

,rouge-1,rouge-2,rouge-4,rouge-l,rouge-w-1.2,rouge-s4,rouge-su4
count,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
mean,0.284238,0.093417,0.055448,0.187455,0.074975,0.085250,0.119377
std,0.087090,0.078919,0.064825,0.069680,0.031805,0.068527,0.071039
min,0.193833,0.008889,0.000000,0.116505,0.043291,0.018100,0.048120
25%,0.202649,0.016202,0.000000,0.123348,0.044174,0.022716,0.055335
50%,0.277073,0.085406,0.030769,0.176964,0.067746,0.068426,0.104178
75%,0.348191,0.157903,0.116917,0.235411,0.102984,0.140855,0.169691
max,0.434783,0.219298,0.151786,0.321739,0.119850,0.203571,0.243323


In [ ]:
mT5_rouge_df = pd.DataFrame.from_dict(mT5_rouge_dict)

for column in mT5_rouge_df:
    mT5_rouge_df[column] = mT5_rouge_df[column].apply(lambda x: x['f'])

mT5_rouge_df.head()
mT5_rouge_df.describe()

,rouge-1,rouge-2,rouge-4,rouge-l,rouge-w-1.2,rouge-s4,rouge-su4
count,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
mean,0.214794,0.049672,0.010313,0.124378,0.044798,0.040740,0.070726
std,0.049118,0.029358,0.018713,0.027960,0.009113,0.025987,0.026539
min,0.143713,0.022099,0.000000,0.072289,0.029609,0.015000,0.039419
25%,0.196992,0.024279,0.000000,0.111225,0.039813,0.023923,0.050396
50%,0.207291,0.039106,0.000000,0.128912,0.044158,0.033089,0.061700
75%,0.228261,0.080786,0.009934,0.130969,0.052326,0.048878,0.091977
max,0.305732,0.090323,0.044944,0.176796,0.058165,0.085393,0.110075
